In [1]:
import pandas as pd
import numpy as np
%matplotlib inline

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
import glob

In [4]:
data = []
list = [pd.read_json(filename, lines=True) for filename in glob.glob("../week4/data/return-data/*.txt")]
df = pd.concat(list, axis=0)
df.head()

,basket,returnLabel,totalAmount,transactionId,zipCode
0,"[3, 1]",0,106,3520406327,7667
1,[1],0,50,6030070379,7647
2,"[1, 1, 5, 4, 1]",0,175,6579182382,2633
3,"[1, 2, 0, 1]",0,272,2779338441,4309
4,[2],0,14,3198368336,4659


In [5]:
df.drop(['transactionId','zipCode'],axis=1,inplace=True)
df.head()

,basket,returnLabel,totalAmount
0,"[3, 1]",0,106
1,[1],0,50
2,"[1, 1, 5, 4, 1]",0,175
3,"[1, 2, 0, 1]",0,272
4,[2],0,14


In [6]:
def categories(c, value):
    return c.count(value)

df['c_0'] = df['basket'].apply(lambda x: categories(x, 0))
df['c_1'] = df['basket'].apply(lambda x: categories(x, 1))
df['c_2'] = df['basket'].apply(lambda x: categories(x, 2))
df['c_3'] = df['basket'].apply(lambda x: categories(x, 3))
df['c_4'] = df['basket'].apply(lambda x: categories(x, 4))
df['c_5'] = df['basket'].apply(lambda x: categories(x, 5))

df.drop(['basket'],axis=1,inplace=True)

df.head()

,returnLabel,totalAmount,c_0,c_1,c_2,c_3,c_4,c_5
0,0,106,0,1,0,1,0,0
1,0,50,0,1,0,0,0,0
2,0,175,0,3,0,0,1,1
3,0,272,1,2,1,0,0,0
4,0,14,0,0,1,0,0,0


In [7]:
from sklearn import preprocessing

def normalize(data, target):
    norm = preprocessing.normalize(data, axis=0)
    df_concat = np.concatenate((target, norm), axis=1)

    return pd.DataFrame(df_concat)

In [8]:
df_norm = normalize(df[['totalAmount', 'c_0', 'c_1', 'c_2', 'c_3', 'c_4', 'c_5']], df[['returnLabel']])
df_norm.columns = ['returnLabel', 'totalAmount', 'c_0', 'c_1', 'c_2', 'c_3', 'c_4', 'c_5']
df_norm.head()

,returnLabel,totalAmount,c_0,c_1,c_2,c_3,c_4,c_5
0,0.0,0.001905,0.000000,0.004752,0.00000,0.004611,0.000000,0.00000
1,0.0,0.000899,0.000000,0.004752,0.00000,0.000000,0.000000,0.00000
2,0.0,0.003145,0.000000,0.014257,0.00000,0.000000,0.004447,0.00481
3,0.0,0.004889,0.004764,0.009505,0.00473,0.000000,0.000000,0.00000
4,0.0,0.000252,0.000000,0.000000,0.00473,0.000000,0.000000,0.00000


In [9]:
from sklearn.model_selection import train_test_split

X = df_norm.drop(['returnLabel'],axis=1)
y = df_norm['returnLabel']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30,random_state=101)

In [10]:
X_test_norm = preprocessing.normalize(X_test, axis=0)
X_test_norm

array([[0.01735141, 0.        , 0.01727801, ..., 0.00838886, 0.01651934,
        0.01774224],
       [0.01126205, 0.00871048, 0.        , ..., 0.00838886, 0.03303868,
        0.01774224],
       [0.00196431, 0.02613145, 0.00863901, ..., 0.00838886, 0.        ,
        0.        ],
       ...,
       [0.01080371, 0.00871048, 0.01727801, ..., 0.00838886, 0.        ,
        0.01774224],
       [0.01735141, 0.01742097, 0.01727801, ..., 0.        , 0.00825967,
        0.        ],
       [0.00903583, 0.00871048, 0.00863901, ..., 0.        , 0.        ,
        0.01774224]])

In [11]:
X_train_norm = preprocessing.normalize(X_train, axis=0)
X_train_norm

array([[0.00092461, 0.        , 0.00569083, ..., 0.        , 0.        ,
        0.        ],
       [0.00146218, 0.        , 0.        , ..., 0.00551947, 0.        ,
        0.        ],
       [0.00021503, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.00894509, 0.01138258, 0.01138165, ..., 0.        , 0.01055589,
        0.        ],
       [0.0041285 , 0.00569129, 0.        , ..., 0.01103895, 0.        ,
        0.        ],
       [0.01182644, 0.        , 0.01138165, ..., 0.01103895, 0.01055589,
        0.00572504]])

In [12]:
import math

def euclideanDistance(feature1, feature2):
    distance = 0
    for i in range(len(feature1)):
        distance += pow((feature1[i] - feature2[i]), 2)
    return math.sqrt(distance)

In [13]:
import operator

def getNeighbors(trainset, testfeature, k):
    distances = []
    
    length = len(X_test) - 1
    for i in range(len(trainset)):
        distance = euclideanDistance(testfeature, trainset[i])
        distances.append((trainset[i], distance, i))
    distances.sort(key=operator.itemgetter(1))
    neighbors = []
    for i in range(k):
        neighbors.append(distances[i][2])
    return neighbors

In [14]:
def customNeighborClassifier(trainset, testset, predictions, k):
    for i in range(len(testset)):
        predictions.append(getNeighbors(trainset, testset[i], k))
    return predictions

In [15]:
predictions = []

neighbor = customNeighborClassifier(X_train_norm, X_test_norm, predictions, 5)
neighbor

[[20988, 18239, 3980, 6493, 8902],
 [1663, 1360, 66, 3191, 14255],
 [7589, 3971, 17501, 20773, 20351],
 [1909, 18399, 21245, 14038, 38],
 [8157, 4009, 1154, 14175, 11476],
 [5469, 18614, 21360, 10214, 2331],
 [2911, 14978, 16671, 12317, 12655],
 [16009, 16295, 3506, 11132, 2019],
 [17298, 15302, 3797, 15029, 2620],
 [11918, 14484, 19238, 5659, 7011],
 [9316, 20, 14277, 219, 20262],
 [20084, 12548, 13767, 15068, 8825],
 [13738, 448, 2452, 5716, 5240],
 [20501, 21512, 18351, 11928, 11051],
 [20141, 18158, 3335, 21451, 7036],
 [6900, 2277, 347, 4869, 7617],
 [21165, 16081, 12343, 874, 21302],
 [16785, 19229, 11191, 8866, 3639],
 [3338, 17979, 19017, 14732, 3603],
 [5762, 3987, 9780, 20777, 2885],
 [1820, 11282, 12872, 8012, 13864],
 [6821, 15998, 2600, 2620, 11757],
 [14678, 7196, 9245, 12604, 19872],
 [10864, 4529, 1021, 13990, 5761],
 [7819, 3282, 10051, 2158, 14964],
 [9775, 3377, 16605, 6405, 18161],
 [4018, 19117, 1540, 19918, 8367],
 [9430, 687, 18106, 1928, 4447],
 [15971, 14673, 1